In [8]:
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

In [9]:
z = torch.zeros((3, 256, 256))
z

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [66]:
class Cutout(object):
    """
    Randomly mask out one or more patches from an image.
    https://github.com/uoguelph-mlrg/Cutout

    Parameters
    ----------
    img_size : int
        The size of the images to expect.
    n_holes : int
        Number of patches to cut out of each image.
    length : int
        The length (in pixels) of each square patch.
    n_masks : int
        Pre-build N masks to randomly choose from (so they don't have to be generated at runtime).
    """
    def __init__(self, img_size, n_holes=1, length=8, n_masks=5000):
        self.img_size = img_size
        self.n_holes = n_holes
        self.length = length
        self.n_masks = n_masks

        self.masks = self.get_masks(self.n_masks)

    def get_masks(self, n):
        masks = torch.zeros((n, self.img_size, self.img_size))
        for i in range(n):
            masks[i, :, :] = self.random_mask()
        return masks

    def random_mask(self):
        h = self.img_size
        w = self.img_size

        mask = np.ones((h, w), np.float32)

        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1: y2, x1: x2] = 0.

        mask = torch.from_numpy(mask)
        return mask

    def __call__(self, img):
        """
        Paramters
        ---------
        img : `torch.Tensor`
            Tensor image of size (C, H, W).

        Returns
        -------
        `torch.Tensor`
            Image with n_holes of dimension length x length cut out of it.
        """
        mask_idx = np.random.choice(self.masks.shape[0])
        mask = self.masks[mask_idx]
        mask = mask.expand_as(img)
        img = img * mask

        return img

In [67]:
%%time
ct = Cutout(256, length=256 // 8)

CPU times: user 11.5 s, sys: 1.23 s, total: 12.8 s
Wall time: 887 ms


In [69]:
%%timeit -n 100
ct(z)

74.3 µs ± 21.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
256 ** 2

65536

In [1]:
??torch.optim.SGD

Object `torch.optim.SGD` not found.
